In [1]:
from IPython.display import FileLink

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(DESeq2)
library(phyloseq)
library(plyr); library(dplyr)
library(ggplot2)
library(doParallel)
library(foreach)
library(reshape)
library(RColorBrewer)
library(gridExtra)
library(stats)
library(data.table)
library(compare, lib.loc="/home/ashley/R/x86_64-pc-linux-gnu-library/3.1/")
library(lawstat, lib.loc="/home/ashley/R/x86_64-pc-linux-gnu-library/3.1/")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:stats’:

    xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, intersect,
    is.unsorted, lapply, Map, mapply, match, mget, order, paste, pmax,
    pmax.int, pmin, pmin.int, Position, rank, rbind, Reduce, rep.int,
    rownames, sapply, setdiff, sort, table, tapply, union, unique,
    unlist, unsplit

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package:

###This is the file from the [Sparsity Notebook](./Sparsity_make_l2fc_table.ipynb).

In [4]:
%%R
df_all = read.csv("/var/seq_data/priming_exp/data/l2fc_table.csv")
head(df_all)

  sparsity_threshold Day Treatment       OTU log2FoldChange         p    Rank1
1               0.25  14    13C000  OTU.4204    -0.20553170 1.0000000 Bacteria
2               0.25  14    13C000   OTU.467    -0.33054855 1.0000000 Bacteria
3               0.25  14    13C000   OTU.905     0.01698885 0.9827594 Bacteria
4               0.25  14    13C000  OTU.7253     0.02799601 0.9772968 Bacteria
5               0.25  14    13C000    OTU.67    -0.60115954 1.0000000 Bacteria
6               0.25  14    13C000 OTU.10580     0.36193427 0.7248674 Bacteria
          Rank2 Rank3                Rank4 Rank5 Rank6 Rank7 Rank8 padj
1 Acidobacteria DA023                 <NA>  <NA>  <NA>  <NA>  <NA>    1
2 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
3 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
4 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
5 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
6 Acidobacteria

In [5]:
%%R
nrow (df_all)

[1] 28471


### Treatment: CC

In [6]:
%%R
FDR = 0.10
df_CC_resp = subset(df_all, (Treatment == "13C000")&(padj <= FDR))
head(df_CC_resp)

    sparsity_threshold Day Treatment       OTU log2FoldChange            p
134               0.25  14    13C000  OTU.7337       2.477516 0.0018908765
251               0.25  14    13C000   OTU.950       3.497786 0.0021470637
252               0.25  14    13C000 OTU.11380       5.169283 0.0002911960
263               0.25  14    13C000  OTU.5228       2.654885 0.0002285059
269               0.25  14    13C000   OTU.903       3.807659 0.0015749699
270               0.25  14    13C000   OTU.752       4.813177 0.0002952637
       Rank1           Rank2            Rank3              Rank4
134 Bacteria   Acidobacteria       Holophagae             iii1-8
251 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
252 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
263 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
269 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
270 Bacteria Verrucomicrobia   Spartobacteria Chthoniobacterales
                   R

In [7]:
%%R
phy.sum.day.CC = df_CC_resp %>%
    select(OTU, Day, Treatment, Rank2, Rank3) %>%
    group_by(Day, Rank2, Rank3, Treatment) %>%
    summarize(Counts = n()) %>%
    arrange(desc(Counts)) 
    
phy.sum.day.CC

Source: local data frame [83 x 5]
Groups: Day, Rank2, Rank3

   Day           Rank2                          Rank3 Treatment Counts
1   14   Acidobacteria Candidatus_Chloracidobacterium    13C000      1
2   14   Acidobacteria                     Holophagae    13C000      1
3   14   Acidobacteria           Order_Incertae_Sedis    13C000      1
4   14   Acidobacteria                        SJA-149    13C000      1
5   14  Actinobacteria                  Micrococcales    13C000      1
6   14  Actinobacteria              Pseudonocardiales    13C000      1
7   14 Armatimonadetes                  Armatimonadia    13C000      1
8   14 Armatimonadetes           uncultured_bacterium    13C000      3
9   14   Bacteroidetes                     Cytophagia    13C000     13
10  14   Bacteroidetes                  Flavobacteria    13C000     10
.. ...             ...                            ...       ...    ...


### Treatment: 100

In [8]:
%%R
FDR = 0.10
df_100_resp = subset(df_all, (Treatment == "13C100")&(padj <= FDR))
nrow (df_100_resp)

[1] 416


In [9]:
%%R
phy.sum.day.C100 = df_100_resp %>%
    select(OTU, Day, Treatment, Rank2, Rank3) %>%
    group_by(Day, Rank2, Rank3, Treatment) %>%
    summarize(Counts = n()) %>%
    arrange(desc(Counts)) 
    
phy.sum.day.C100

Source: local data frame [71 x 5]
Groups: Day, Rank2, Rank3

   Day                   Rank2                Rank3 Treatment Counts
1   14          Actinobacteria    Corynebacteriales    13C100      1
2   14          Actinobacteria        Micrococcales    13C100      1
3   14          Actinobacteria     Streptomycetales    13C100      1
4   14           Bacteroidetes           Cytophagia    13C100      8
5   14           Bacteroidetes     Sphingobacteriia    13C100      2
6   14 Candidate_division_BRC1 uncultured_bacterium    13C100      2
7   14                Chlorobi            Chlorobia    13C100      1
8   14             Chloroflexi         Anaerolineae    13C100      1
9   14             Chloroflexi    Herpetosiphonales    13C100      5
10  14             Chloroflexi      Ktedonobacteria    13C100      1
.. ...                     ...                  ...       ...    ...


### Treatment: 700

In [10]:
%%R
FDR = 0.10
df_700_resp = subset(df_all, (Treatment == "13C700")&(padj <= FDR))
nrow (df_700_resp)

[1] 524


In [11]:
%%R
phy.sum.day.C700 = df_700_resp %>%
    select(OTU, Day, Treatment, Rank2, Rank3) %>%
    group_by(Day, Rank2, Rank3, Treatment) %>%
    summarize(Counts = n()) %>%
    arrange(desc(Counts)) 
    
phy.sum.day.C700

Source: local data frame [85 x 5]
Groups: Day, Rank2, Rank3

   Day           Rank2                          Rank3 Treatment Counts
1   14   Acidobacteria                          11-24    13C700      1
2   14   Acidobacteria Candidatus_Chloracidobacterium    13C700      1
3   14   Acidobacteria                           RB25    13C700      1
4   14   Acidobacteria                        SJA-149    13C700      1
5   14  Actinobacteria                  Micrococcales    13C700      1
6   14 Armatimonadetes                  Armatimonadia    13C700      1
7   14 Armatimonadetes           uncultured_bacterium    13C700      2
8   14   Bacteroidetes                     Cytophagia    13C700     24
9   14   Bacteroidetes                  Flavobacteria    13C700      9
10  14   Bacteroidetes               Sphingobacteriia    13C700     60
.. ...             ...                            ...       ...    ...


In [12]:
%%R
phy.counts.class = rbind(phy.sum.day.CC, phy.sum.day.C100, phy.sum.day.C700)
phy.counts.class

Source: local data frame [239 x 5]
Groups: Day, Rank2, Rank3

   Day           Rank2                          Rank3 Treatment Counts
1   14   Acidobacteria Candidatus_Chloracidobacterium    13C000      1
2   14   Acidobacteria                     Holophagae    13C000      1
3   14   Acidobacteria           Order_Incertae_Sedis    13C000      1
4   14   Acidobacteria                        SJA-149    13C000      1
5   14  Actinobacteria                  Micrococcales    13C000      1
6   14  Actinobacteria              Pseudonocardiales    13C000      1
7   14 Armatimonadetes                  Armatimonadia    13C000      1
8   14 Armatimonadetes           uncultured_bacterium    13C000      3
9   14   Bacteroidetes                     Cytophagia    13C000     13
10  14   Bacteroidetes                  Flavobacteria    13C000     10
.. ...             ...                            ...       ...    ...


In [13]:
%%R

write.table(phy.counts.class, 
            file = "data/class_counts.csv",
            row.names = FALSE,
            sep = ",")

In [16]:
%%R
FDR = 0.10

topN = df_all %>%
    filter(padj <= FDR) %>%
    group_by(Treatment, Day) %>%
    arrange(desc(log2FoldChange)) %>%
    #group_by(log2FoldChange) %>%
    top_n(log2FoldChange, n=50)

topN

Source: local data frame [450 x 15]
Groups: Treatment, Day

   sparsity_threshold Day Treatment      OTU log2FoldChange            p
1                0.25  14    13C000 OTU.1647       7.376202 2.036806e-07
2                0.25  14    13C000  OTU.385       7.033637 3.303388e-06
3                0.25  14    13C000 OTU.2004       6.979512 5.223504e-07
4                0.25  14    13C000  OTU.690       6.933718 5.295957e-10
5                0.25  14    13C000  OTU.131       6.700015 4.074329e-13
6                0.25  14    13C000 OTU.2418       6.662350 2.846601e-06
7                0.25  14    13C000 OTU.3165       6.609019 3.248237e-13
8                0.25  14    13C000 OTU.1590       6.596085 5.355882e-08
9                0.25  14    13C000   OTU.31       6.561500 1.853986e-12
10               0.25  14    13C000  OTU.151       6.439481 3.686976e-14
..                ... ...       ...      ...            ...          ...
Variables not shown: Rank1 (fctr), Rank2 (fctr), Rank3 (fctr), R

In [17]:
%%R

write.table(topN, 
            file = "data/top150_responders.csv",
            row.names = FALSE,
            sep = ",")